# Experiment 003: Pre-optimized Baseline with Multi-Phase Optimization

Using chistyakov/submission_best.csv as starting point (score: 70.93, no overlaps).
Target: 68.92 | Gap: only 2.01 points!

In [1]:
import numpy as np
import pandas as pd
from decimal import Decimal, getcontext
from shapely import affinity
from shapely.geometry import Polygon
from shapely.ops import unary_union
from shapely.strtree import STRtree
from scipy.spatial import ConvexHull
from scipy.optimize import minimize_scalar

getcontext().prec = 30
scale_factor = Decimal('1e15')

print("Libraries loaded")

Libraries loaded


In [2]:
class ChristmasTree:
    def __init__(self, center_x='0', center_y='0', angle='0'):
        self.center_x = Decimal(str(center_x))
        self.center_y = Decimal(str(center_y))
        self.angle = Decimal(str(angle))

        trunk_w = Decimal('0.15')
        trunk_h = Decimal('0.2')
        base_w = Decimal('0.7')
        mid_w = Decimal('0.4')
        top_w = Decimal('0.25')
        tip_y = Decimal('0.8')
        tier_1_y = Decimal('0.5')
        tier_2_y = Decimal('0.25')
        base_y = Decimal('0.0')
        trunk_bottom_y = -trunk_h

        initial_polygon = Polygon([
            (Decimal('0.0') * scale_factor, tip_y * scale_factor),
            (top_w / Decimal('2') * scale_factor, tier_1_y * scale_factor),
            (top_w / Decimal('4') * scale_factor, tier_1_y * scale_factor),
            (mid_w / Decimal('2') * scale_factor, tier_2_y * scale_factor),
            (mid_w / Decimal('4') * scale_factor, tier_2_y * scale_factor),
            (base_w / Decimal('2') * scale_factor, base_y * scale_factor),
            (trunk_w / Decimal('2') * scale_factor, base_y * scale_factor),
            (trunk_w / Decimal('2') * scale_factor, trunk_bottom_y * scale_factor),
            (-(trunk_w / Decimal('2')) * scale_factor, trunk_bottom_y * scale_factor),
            (-(trunk_w / Decimal('2')) * scale_factor, base_y * scale_factor),
            (-(base_w / Decimal('2')) * scale_factor, base_y * scale_factor),
            (-(mid_w / Decimal('4')) * scale_factor, tier_2_y * scale_factor),
            (-(mid_w / Decimal('2')) * scale_factor, tier_2_y * scale_factor),
            (-(top_w / Decimal('4')) * scale_factor, tier_1_y * scale_factor),
            (-(top_w / Decimal('2')) * scale_factor, tier_1_y * scale_factor),
        ])
        rotated = affinity.rotate(initial_polygon, float(self.angle), origin=(0, 0))
        self.polygon = affinity.translate(rotated,
                                          xoff=float(self.center_x * scale_factor),
                                          yoff=float(self.center_y * scale_factor))

    def clone(self):
        return ChristmasTree(str(self.center_x), str(self.center_y), str(self.angle))

print("ChristmasTree class defined")

ChristmasTree class defined


In [3]:
def get_tree_list_side_length(tree_list):
    all_polygons = [t.polygon for t in tree_list]
    bounds = unary_union(all_polygons).bounds
    return Decimal(max(bounds[2] - bounds[0], bounds[3] - bounds[1])) / scale_factor

def get_score(trees, n):
    if not trees:
        return 0.0
    side = get_tree_list_side_length(trees)
    return float(side ** 2 / Decimal(n))

def has_overlap(trees):
    if len(trees) <= 1:
        return False
    polygons = [t.polygon for t in trees]
    tree_index = STRtree(polygons)
    for i, poly in enumerate(polygons):
        indices = tree_index.query(poly)
        for idx in indices:
            if idx == i:
                continue
            if poly.intersects(polygons[idx]) and not poly.touches(polygons[idx]):
                return True
    return False

def load_configuration_from_df(n, df):
    group_data = df[df["id"].str.startswith(f"{n:03d}_")]
    trees = []
    for _, row in group_data.iterrows():
        x = str(row["x"])[1:] if str(row["x"]).startswith('s') else str(row["x"])
        y = str(row["y"])[1:] if str(row["y"]).startswith('s') else str(row["y"])
        deg = str(row["deg"])[1:] if str(row["deg"]).startswith('s') else str(row["deg"])
        if x and y and deg:
            trees.append(ChristmasTree(x, y, deg))
    return trees

def score_submission(file_path, max_n=200):
    df = pd.read_csv(file_path)
    total_score = 0.0
    failed_overlap_n = []
    
    for n in range(1, max_n + 1):
        trees = load_configuration_from_df(n, df)
        if trees:
            current_score = get_score(trees, n)
            total_score += current_score
            if has_overlap(trees):
                failed_overlap_n.append(n)
    
    return total_score, failed_overlap_n

print("Helper functions defined")

Helper functions defined


In [4]:
# Verify the pre-optimized baseline
print("Scoring chistyakov/submission_best.csv baseline...")
baseline_score, baseline_overlaps = score_submission('/home/code/experiments/003_preoptimized/baseline.csv')
print(f"Baseline score: {baseline_score:.6f}")
print(f"Overlaps: {baseline_overlaps if baseline_overlaps else 'None'}")
print(f"Gap to target (68.92): {baseline_score - 68.922808:.6f} points")

Scoring chistyakov/submission_best.csv baseline...


Baseline score: 70.926150
Overlaps: None
Gap to target (68.92): 2.003342 points


In [5]:
# Also check other pre-optimized baselines for comparison
print("\nChecking other pre-optimized baselines...")

baselines = [
    '/home/nonroot/snapshots/santa-2025/21116303805/code/preoptimized/santa-2025.csv',
    '/home/nonroot/snapshots/santa-2025/21116303805/code/preoptimized/telegram/71.97.csv',
    '/home/nonroot/snapshots/santa-2025/21116303805/code/preoptimized/telegram/72.49.csv',
    '/home/nonroot/snapshots/santa-2025/21116303805/code/preoptimized/ensemble.csv',
]

import os
for path in baselines:
    if os.path.exists(path):
        try:
            score, overlaps = score_submission(path)
            print(f"{os.path.basename(path)}: score={score:.6f}, overlaps={len(overlaps)}")
        except Exception as e:
            print(f"{os.path.basename(path)}: Error - {e}")
    else:
        print(f"{os.path.basename(path)}: Not found")


Checking other pre-optimized baselines...


santa-2025.csv: score=70.676102, overlaps=12


71.97.csv: score=71.972027, overlaps=0


72.49.csv: score=72.495739, overlaps=0


ensemble.csv: score=70.676102, overlaps=13


In [6]:
# Try to repair santa-2025.csv overlaps using chistyakov configurations
print("Repairing santa-2025.csv overlaps...")

# Load both CSVs
df_santa = pd.read_csv('/home/nonroot/snapshots/santa-2025/21116303805/code/preoptimized/santa-2025.csv')
df_chistyakov = pd.read_csv('/home/code/experiments/003_preoptimized/baseline.csv')

# Find which N values have overlaps in santa-2025.csv
overlap_n = []
for n in range(1, 201):
    trees = load_configuration_from_df(n, df_santa)
    if trees and has_overlap(trees):
        overlap_n.append(n)

print(f"Overlapping N values in santa-2025.csv: {overlap_n}")

# Replace overlapping configurations with chistyakov configurations
df_repaired = df_santa.copy()

for n in overlap_n:
    # Remove old rows for this N
    mask = df_repaired['id'].str.startswith(f'{n:03d}_')
    df_repaired = df_repaired[~mask]
    
    # Add rows from chistyakov
    chistyakov_rows = df_chistyakov[df_chistyakov['id'].str.startswith(f'{n:03d}_')]
    df_repaired = pd.concat([df_repaired, chistyakov_rows], ignore_index=True)

# Sort by id
df_repaired = df_repaired.sort_values('id').reset_index(drop=True)

# Save repaired CSV
df_repaired.to_csv('/home/code/experiments/003_preoptimized/repaired_baseline.csv', index=False)

# Score the repaired version
repaired_score, repaired_overlaps = score_submission('/home/code/experiments/003_preoptimized/repaired_baseline.csv')
print(f"\nRepaired baseline score: {repaired_score:.6f}")
print(f"Overlaps: {repaired_overlaps if repaired_overlaps else 'None'}")
print(f"Gap to target: {repaired_score - 68.922808:.6f} points")

Repairing santa-2025.csv overlaps...


Overlapping N values in santa-2025.csv: [30, 41, 43, 61, 77, 110, 114, 117, 134, 137, 170, 171]



Repaired baseline score: 70.682741
Overlaps: None
Gap to target: 1.759933 points


In [7]:
# Use the repaired baseline as starting point
import shutil
shutil.copy('/home/code/experiments/003_preoptimized/repaired_baseline.csv', '/home/code/experiments/003_preoptimized/submission.csv')
print(f"Starting optimization from repaired baseline: {repaired_score:.6f}")
print(f"Gap to target: {repaired_score - 68.922808:.6f} points")

Starting optimization from repaired baseline: 70.682741
Gap to target: 1.759933 points


In [8]:
# Apply fix_direction rotation tightening to each N configuration
from scipy.spatial import ConvexHull
from scipy.optimize import minimize_scalar

def calculate_bbox_side_at_angle(angle_deg, points):
    angle_rad = np.radians(angle_deg)
    c, s = np.cos(angle_rad), np.sin(angle_rad)
    rot_matrix_T = np.array([[c, s], [-s, c]])
    rotated_points = points.dot(rot_matrix_T)
    min_xy = np.min(rotated_points, axis=0)
    max_xy = np.max(rotated_points, axis=0)
    return max(max_xy[0] - min_xy[0], max_xy[1] - min_xy[1])

def optimize_rotation(trees):
    all_points = []
    for tree in trees:
        coords = list(tree.polygon.exterior.coords)
        all_points.extend([(float(c[0]) / float(scale_factor), float(c[1]) / float(scale_factor)) for c in coords])
    points_np = np.array(all_points)
    hull_points = points_np[ConvexHull(points_np).vertices]
    initial_side = calculate_bbox_side_at_angle(0, hull_points)
    res = minimize_scalar(lambda a: calculate_bbox_side_at_angle(a, hull_points),
                          bounds=(0.001, 89.999), method='bounded')
    found_angle_deg = res.x
    found_side = res.fun
    improvement = initial_side - found_side
    if improvement > 1e-8:
        return Decimal(str(found_side)), found_angle_deg
    else:
        return Decimal(str(initial_side)), 0.0

def apply_rotation(trees, angle_deg):
    if not trees or abs(angle_deg) < 1e-9:
        return [t.clone() for t in trees]
    bounds = [t.polygon.bounds for t in trees]
    min_x = min(b[0] for b in bounds)
    min_y = min(b[1] for b in bounds)
    max_x = max(b[2] for b in bounds)
    max_y = max(b[3] for b in bounds)
    rotation_center = np.array([(min_x + max_x) / 2.0, (min_y + max_y) / 2.0])
    angle_rad = np.radians(angle_deg)
    c, s = np.cos(angle_rad), np.sin(angle_rad)
    rot_matrix = np.array([[c, -s], [s, c]])
    points = np.array([[float(t.center_x * scale_factor), float(t.center_y * scale_factor)] for t in trees])
    shifted = points - rotation_center
    rotated = shifted.dot(rot_matrix.T) + rotation_center
    rotated_trees = []
    for i in range(len(trees)):
        new_x = Decimal(str(rotated[i, 0])) / scale_factor
        new_y = Decimal(str(rotated[i, 1])) / scale_factor
        new_angle = trees[i].angle + Decimal(str(angle_deg))
        new_tree = ChristmasTree(new_x, new_y, new_angle)
        rotated_trees.append(new_tree)
    return rotated_trees

print("Rotation functions defined")

Rotation functions defined


In [9]:
# Load the repaired baseline and apply fix_direction to each N
print("Applying fix_direction to repaired baseline...")

df = pd.read_csv('/home/code/experiments/003_preoptimized/repaired_baseline.csv')

optimized_configs = {}
total_improvement = 0

for n in range(1, 201):
    trees = load_configuration_from_df(n, df)
    if trees:
        old_score = get_score(trees, n)
        best_side, best_angle = optimize_rotation(trees)
        if abs(best_angle) > 1e-9:
            trees = apply_rotation(trees, best_angle)
        new_score = get_score(trees, n)
        improvement = old_score - new_score
        total_improvement += improvement
        optimized_configs[n] = trees
        if improvement > 0.001:
            print(f"N={n}: improved by {improvement:.6f}")

print(f"\nTotal improvement from fix_direction: {total_improvement:.6f}")

# Calculate new total score
new_total_score = sum(get_score(trees, n) for n, trees in optimized_configs.items())
print(f"New total score: {new_total_score:.6f}")
print(f"Gap to target: {new_total_score - 68.922808:.6f} points")

Applying fix_direction to repaired baseline...



Total improvement from fix_direction: 0.000000


New total score: 70.682741
Gap to target: 1.759933 points


In [10]:
# Implement backward propagation from smartmanoj kernel
# For N=200 down to N=2, try removing boundary-touching trees
# and check if resulting (N-1) config is better than current best

def get_boundary_trees(trees):
    """Find trees that touch the bounding box boundary."""
    all_polygons = [t.polygon for t in trees]
    bounds = unary_union(all_polygons).bounds
    gx0, gy0, gx1, gy1 = bounds
    eps = 0.01 * float(scale_factor)
    
    boundary_indices = []
    for i, tree in enumerate(trees):
        tb = tree.polygon.bounds
        if (abs(tb[0] - gx0) < eps or abs(tb[2] - gx1) < eps or
            abs(tb[1] - gy0) < eps or abs(tb[3] - gy1) < eps):
            boundary_indices.append(i)
    return boundary_indices

print("Backward propagation functions defined")

Backward propagation functions defined


In [11]:
# Run backward propagation
print("Running backward propagation...")
print("Starting from repaired baseline score: 70.682741")

# Load current best configurations
df = pd.read_csv('/home/code/experiments/003_preoptimized/repaired_baseline.csv')
best_configs = {}
for n in range(1, 201):
    trees = load_configuration_from_df(n, df)
    if trees:
        best_configs[n] = trees

# Calculate current scores
best_scores = {n: get_score(trees, n) for n, trees in best_configs.items()}

improvements = 0

# Backward propagation: from N=200 down to N=2
for n in range(200, 1, -1):
    if n not in best_configs:
        continue
    
    trees_n = best_configs[n]
    boundary_indices = get_boundary_trees(trees_n)
    
    # Try removing each boundary tree
    for idx in boundary_indices:
        # Create N-1 configuration by removing this tree
        trees_n_minus_1 = [t.clone() for i, t in enumerate(trees_n) if i != idx]
        
        if len(trees_n_minus_1) != n - 1:
            continue
        
        # Check for overlaps
        if has_overlap(trees_n_minus_1):
            continue
        
        # Calculate score for N-1
        new_score = get_score(trees_n_minus_1, n - 1)
        
        # Compare with current best for N-1
        if n - 1 in best_scores:
            if new_score < best_scores[n - 1]:
                improvement = best_scores[n - 1] - new_score
                print(f"N={n-1}: improved by {improvement:.6f} (from {best_scores[n-1]:.6f} to {new_score:.6f})")
                best_configs[n - 1] = trees_n_minus_1
                best_scores[n - 1] = new_score
                improvements += 1

print(f"\nTotal improvements: {improvements}")

# Calculate new total score
new_total_score = sum(best_scores.values())
print(f"New total score: {new_total_score:.6f}")
print(f"Gap to target: {new_total_score - 68.922808:.6f} points")

Running backward propagation...
Starting from repaired baseline score: 70.682741


N=137: improved by 0.001737 (from 0.345264 to 0.343528)


N=137: improved by 0.000000 (from 0.343528 to 0.343528)
N=137: improved by 0.000000 (from 0.343528 to 0.343528)



Total improvements: 3
New total score: 70.681004
Gap to target: 1.758196 points
